In [1]:
import pandas as pd
import numpy as np
import os
import glob
from power_to_temperature import *
from test import *

In [2]:
freqs,vals,bwidth = bandwidth(50.,90.)
Freqs = np.linspace(1e-22,250,32769)
eta_nu = eta(Freqs)
mask = (Freqs>=50)&(Freqs<=91)
bwidth=1.

In [3]:
folders = glob.glob('Data/*')
folders.sort()
i=-1
for subdirs, dirs, files in os.walk('Data'):
    dirs.sort()
    files.sort()
    K_jnc,measure = [],[]
    K_date,measure_date=[],[]

    for file in files:
        path = os.path.join(subdirs,file)
        date = file.split("_")[0]
        if  os.path.getsize(path)==0:
            print 'EMPTY FILE:',path
            continue
        elif file.endswith('short.dat'):
            dB_short = np.loadtxt(path,unpack=True)
            T_short = Res2Temp(dB_short,bwidth)
        elif file.endswith('50ohm.dat'):
            dB_50ohm = np.loadtxt(path,unpack=True)
            T_50ohm = Res2Temp(dB_50ohm,bwidth)
        elif file.endswith('noise.dat'):
            dB_noise = np.loadtxt(path,unpack=True)
        elif file.endswith('antenna.dat'):
            dB_antenna = np.loadtxt(path,unpack=True)
            dB_clean = dB_antenna - dB_noise - 93.0
            T_antenna = Radio_source_trans(dB_clean, Freqs, bwidth)
            Tamb = round(np.genfromtxt(path,comments='!',skip_header= 18,max_rows=1)[1]+273.15,2)
            Kjnc = Tamb/(T_50ohm-T_short)
            T_measure = Kjnc * (T_antenna/eta_nu - T_short)
            K_jnc.append(Kjnc),measure.append(T_measure)
            K_date.append(date),measure_date.append(date)
    if i>=0 and i<len(folders) and K_jnc and measure:
        name = os.path.normpath(folders[i])
        name = name.split("/")[1]
        K_jnc = np.transpose(K_jnc)
        measure = np.transpose(measure)
        Kjnc_table = pd.DataFrame(K_jnc[mask], index = Freqs[mask], columns = K_date)
        Kjnc_table.to_hdf('Data_filter/K_jnc/'+name+'.hdf5','df')
        measure_table = pd.DataFrame(measure[mask], index = Freqs[mask], columns = measure_date)
        measure_table.to_hdf('Data_filter/Tmeas_jnc/'+name+'.hdf5','df')
    i+=1
    

/home/hiram/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:29: RuntimeWarning: divide by zero encountered in divide


EMPTY FILE: Data/2013-06-14-02/2013-06-14-02-02-29_Ch_2_antenna.dat
EMPTY FILE: Data/2013-06-14-02/2013-06-14-02-02-39_Ch_2_antenna.dat
EMPTY FILE: Data/2013-06-14-02/2013-06-14-02-02-50_Ch_2_antenna.dat
EMPTY FILE: Data/2013-06-14-02/2013-06-14-02-15-33_Ch_2_antenna.dat
EMPTY FILE: Data/2013-06-14-02/2013-06-14-02-15-44_Ch_2_antenna.dat
EMPTY FILE: Data/2013-06-14-02/2013-06-14-02-15-55_Ch_2_antenna.dat
EMPTY FILE: Data/2013-06-14-02/2013-06-14-02-29-28_Ch_2_antenna.dat
EMPTY FILE: Data/2013-06-14-02/2013-06-14-02-29-39_Ch_2_antenna.dat
EMPTY FILE: Data/2013-06-14-02/2013-06-14-02-29-50_Ch_2_antenna.dat
EMPTY FILE: Data/2013-06-14-02/2013-06-14-02-45-18_Ch_2_antenna.dat
EMPTY FILE: Data/2013-06-14-02/2013-06-14-02-45-29_Ch_2_antenna.dat
EMPTY FILE: Data/2013-06-14-02/2013-06-14-02-45-40_Ch_2_antenna.dat
EMPTY FILE: Data/2013-06-14-03/2013-06-14-03-04-54_Ch_2_antenna.dat
EMPTY FILE: Data/2013-06-14-03/2013-06-14-03-05-05_Ch_2_antenna.dat
EMPTY FILE: Data/2013-06-14-03/2013-06-14-03-05-

IndexError: too many indices for array

In [11]:
day14jnc = glob.glob('Data_filter/Tmeas_jnc/2013-06-14*')
day14jnc.sort()
dfs = []
for path in day14jnc:
    dfs.append(pd.read_hdf(path))
big_frame1 = pd.concat(dfs, axis = 1)
big_frame1.to_hdf('calibration/Day14_jnc.hdf5','df')

In [12]:
day15jnc = glob.glob('Data_filter/Tmeas_jnc/2013-06-15*')
day15jnc.sort()
dfs = []
for path in day15jnc:
    dfs.append(pd.read_hdf(path))
big_frame2 = pd.concat(dfs, axis = 1)
big_frame2.to_hdf('calibration/Day15_jnc.hdf5','df')

In [13]:
all_data = [big_frame1,big_frame2]
day14_15 = pd.concat(all_data,axis=1)
day14_15.to_hdf('calibration/Days14_15_jnc.hdf5','df')

In [4]:
Kday14 = glob.glob('Data_filter/K_jnc/2013-06-14*')
Kday14.sort()
dfs = []
for path in Kday14:
    dfs.append(pd.read_hdf(path))
big_frame1 = pd.concat(dfs, axis = 1)
big_frame1.to_hdf('calibration/Kjnc_Day14.hdf5','df')

In [6]:
Kday15 = glob.glob('Data_filter/K_jnc/2013-06-15*')
Kday15.sort()
dfs = []
for path in Kday15:
    dfs.append(pd.read_hdf(path))
big_frame2 = pd.concat(dfs, axis = 1)
big_frame2.to_hdf('calibration/Kjnc_Day15.hdf5','df')

In [7]:
all_K = [big_frame1,big_frame2]
day14_15 = pd.concat(all_K,axis=1)
day14_15.to_hdf('calibration/Kjnc_Days14_15.hdf5','df')